In [1]:
%reset -f
import gym
import pickle
import tqdm
import numpy as np
import matplotlib.pyplot as plt
from stable_baselines3 import PPO
from stable_baselines3.common.monitor import Monitor
from chameleon import Chameleon
from stable_baselines3.common.callbacks import (
    EvalCallback,
    BaseCallback,
    StopTrainingOnRewardThreshold,
)

In [5]:
env = Chameleon(rtol=0.05)
env = Monitor(env, filename="nbmon")
model = PPO(
    policy="MlpPolicy",
    env=env,
    verbose=0,
)
# model.learn(total_timesteps=int(6e5))
# model.load("logs/best_model.zip")

In [6]:
model.policy

ActorCriticPolicy(
  (features_extractor): FlattenExtractor(
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (mlp_extractor): MlpExtractor(
    (shared_net): Sequential()
    (policy_net): Sequential(
      (0): Linear(in_features=1, out_features=64, bias=True)
      (1): Tanh()
      (2): Linear(in_features=64, out_features=64, bias=True)
      (3): Tanh()
    )
    (value_net): Sequential(
      (0): Linear(in_features=1, out_features=64, bias=True)
      (1): Tanh()
      (2): Linear(in_features=64, out_features=64, bias=True)
      (3): Tanh()
    )
  )
  (action_net): Linear(in_features=64, out_features=1, bias=True)
  (value_net): Linear(in_features=64, out_features=1, bias=True)
)

In [ ]:
# rews = np.loadtxt("nbmon.monitor.csv", delimiter=",", skiprows=2, usecols=(0,))

In [ ]:
# rews.shape

In [ ]:
# rews = rews[865:]

In [ ]:
# rews.shape

In [ ]:
# rews = rews.reshape((1220, 100))
# means = rews.mean(axis=1)
# plt.plot(means)
# plt.title("Episode reward over time")
# plt.xlabel("Epoch (100 episodes)")
# plt.ylabel("Mean reward")
# plt.savefig("rewardvsepoch.png")
# plt.show()

In [ ]:
# model = PPO.load("rtol0.05target0.18nstep6e5")

In [ ]:
# pos_hist = []
# state = env.reset()
# done = False
# while not done:
#     pos_hist.append(env.pos)
#     action = model.predict(state, deterministic=True)[
#         0
#     ]  # need to do deterministic for PPO
#     print(action)
#     state, rew, done, _ = env.step(action)
#     print(rew)

In [ ]:
# pos_hist.append(env.winning_pos)

In [ ]:
# history = []
# for i in range(len(pos_hist)):
#     x_p = pos_hist[i]
#     y = np.zeros(env.n_elems)
#     ar = np.vstack((x_p, y)).T
#     history.append(ar)

# with open("winning_history.txt", "wb") as fp:
#     pickle.dump(history, fp)

In [ ]:
# i = 0
# for pos in pos_hist:
#     i+=1
#     plt.scatter(pos, 0*pos)
#     plt.scatter(0.1, 0)
#     plt.scatter(0.18, 0)
#     plt.title(f"Tip position is {pos[-1]}")
#     plt.savefig(f"frame{i}.png")
#     plt.clf()

## max distance E = 1 agent can reach with the given torques is 0.2. How did i find that? By applying the max torque for each time step until the episode was over.